In [1]:
import pandas as pd
import numpy as np
import pickle
import lightgbm as lgb
from sklearn import  preprocessing,metrics
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('classification_data.csv')
df.head()

,client_id,avg_session_duration,browser,browser_version,campaign,hits,day_of_week,day,device_category,entrances,...,page_depth,page_views,page_views_per_session,sessions,social_network,events,unique_events,week,shopping_stage,frequency
0,GA889580594.1590808701,158.000000,Internet Explorer,11.0,Other,18,5,29,tablet,1,...,18,18,18.000000,1,Other,1,1,22,Window,2
1,GA616880682.1596545776,7.000000,Chrome,79.0.3945.0,Other,7,2,4,desktop,1,...,3,3,3.000000,1,Other,1,1,32,Window,3
2,GA946056279.1598894360,261.333333,Chrome,84.0.4147.135,Other,103,4,31,desktop,3,...,66,102,34.000000,3,Other,1,1,36,Window,2
3,GA609176977.1586186638,126.384615,Chrome,83.0.4103.96,Shoes Sale-Display Ads,42,6,30,mobile,13,...,6,42,3.230769,13,Other,1,1,27,Window,2
4,GA1964696600.1602347357,79.000000,Chrome,85.0.4183.127,Other,13,6,10,tablet,1,...,13,13,13.000000,1,Other,1,1,41,Window,2


In [3]:
# PRE-PROCESSING THE DATA, ENCODING AND SPLITTING IT
df = df.drop(['client_id','browser','browser_version','os'],axis = 1)
le_campaign = preprocessing.LabelEncoder()
df['campaign'] = le_campaign.fit_transform(df['campaign'])
le_device_category = preprocessing.LabelEncoder()
df['device_category'] = le_device_category.fit_transform(df['device_category'])
le_social_network = preprocessing.LabelEncoder()
df['social_network'] = le_social_network.fit_transform(df['social_network'])
le_shopping_stage = preprocessing.LabelEncoder()
df['shopping_stage'] = le_shopping_stage.fit_transform(df['shopping_stage'])

In [4]:
df

,avg_session_duration,campaign,hits,day_of_week,day,device_category,entrances,events_per_session,exits,hour,...,page_depth,page_views,page_views_per_session,sessions,social_network,events,unique_events,week,shopping_stage,frequency
0,158.000000,5,18,5,29,2,1,0.0,1,20,...,18,18,18.000000,1,4,1,1,22,2,2
1,7.000000,5,7,2,4,0,1,1.0,1,5,...,3,3,3.000000,1,4,1,1,32,2,3
2,261.333333,5,103,4,31,0,3,1.0,3,14,...,66,102,34.000000,3,4,1,1,36,2,2
3,126.384615,7,42,6,30,1,13,0.0,13,23,...,6,42,3.230769,13,4,1,1,27,2,2
4,79.000000,5,13,6,10,2,1,0.0,1,7,...,13,13,13.000000,1,4,1,1,41,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98069,114.000000,2,31,5,24,0,1,3.0,1,17,...,28,28,28.000000,1,4,3,3,30,2,2
98070,9.000000,4,3,1,15,1,1,0.0,1,4,...,3,3,3.000000,1,4,1,1,25,2,2
98071,473.500000,5,31,2,25,1,2,3.0,2,20,...,27,28,14.000000,2,4,3,2,35,2,2
98072,2.000000,5,6,0,30,1,2,0.0,2,20,...,3,6,3.000000,2,4,1,1,36,2,2


In [5]:
y = df['shopping_stage']
X = df.drop(['shopping_stage'],axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [6]:
model = lgb.LGBMClassifier()
model.fit(X_train, y_train)

LGBMClassifier()

In [7]:
preds = model.predict(X_test)
print(metrics.accuracy_score(y_test, preds))

0.9999691024254596


In [8]:
pickle.dump(model, open("model.pickle.dat", "wb"))

In [9]:
np.save('campaign.npy', le_campaign.classes_)
np.save('device_category.npy', le_device_category.classes_)
np.save('social_network.npy', le_social_network.classes_)
np.save('shopping_stage.npy', le_shopping_stage.classes_)